<h1>Tarea 3 - Árbol de decisión</h1>
<h4>Universidad autónoma de Baja California<br>
Aprendizaje automático, Dr. Luis Pellegrin<br>
Alan Montesinos Scott</h4>

<p><strong>Descripción: </strong> la actividad consiste en desarrollar el algoritmo id3 bajo ciertas delimitaciones.

Elaborar un reporte que presente el código desarrollado, tomando en cuenta las especificaciones abajo descritas. La extensión del reporte es variable y debe contener los datos del alumno, título de la tarea, y datos de la clase. Además debe presentar una discusión en torno a las diferentes especificaciones solicitadas. Incluir al final del reporte una reflexión personal de la tarea.

<strong>Especificaciones a tomar en cuenta:</strong></p>
<ul>
<li>Utilizar los datos de ejemplo de los ejercicios prácticos revisados en clase: colores y jugar_tenis.</li>
<li>Opcional desplegar gráficamente el árbol resultante.</li>
<li>Programación en lenguaje de programación a decisión del alumno.</li>
<li>Estrategia de programación (iterativo o recursivo) a decisión del alumno.</li>
</ul>


<h4>Introducción</h4>
<p>El algoritmo que se ha implementado es el llamado ID3, creado por J. R. Quinlan, y que emplea un procedimiento de arrriba a abajo haciendo un recorrido voraz por el espacio de las posibles ramifiaciones sin backtracking. Para ello, ID3 hace uso de conceptos como <strong>entropía</strong> y <strong>ganancia de información</strong>.</p>

<h4>Pseudocódigo</h4>

ID3(Ejemplos, Atributo-objetivo, Atributos)
<ol>
    <li>Si todos los Ejemplos son positivos, devolver un nodo etiquetado con True</li>
    <li>Si todos los Ejemplos son negativos, devolver un nodo etiquetado con False</li>
    <li>Si Atributos está vacío, devolver un nodo etiquetado con el valor más frecuente de Atributo-objetivo en Ejemplos. </li>
    <li>En otro caso:
        <ol>
            <li>Sea A el atributo de Atributos que MEJOR clasifica Ejemplos</li>
            <li>Crear Árbol, con un nodo etiquetado con A.</li>
            <li>Para cada posible valor v de A, hacer:
                <ul>
                    <li>Añadir un arco a Árbol, etiquetado con v.</li>
                    <li>Sea Ejemplos(v) el subconjunto de Ejemplos con valor del atributo A igual a v.</li>
                    <li>Si Ejemplos(v) es vacío:
                        <ul>
                            <li>Entonces colocar debajo del arco anterior un nodo etiquetado con el valor más frecuente de Atributo-objetivo en Ejemplos.</li>
                            <li>Si no, colocar debajo del arco anterior el sub árbol ID3(Ejemplos(v), Atributo-objetivo, Atributos-{A}).</li>
                        </ul>
                    </li>
                </ul>
            </li>
            4.4 Devolver Árbol</li>
</ol>

<h4>Código</h4>
<p>Primero importamos las librerias necesarias.</p>

In [1]:
import pandas as pd
import numpy as np

Los conjuntos de datos utilizados son:
<ul>
<li>Jugar_tenis</li>
<li>Colores</li>
</ul>

In [2]:
dataTenis = pd.read_csv("./Jugar_tenis.csv")
dataTenis

,Cielo,Temperatura,Humedad,Viento,Jugar_tenis
0,Sol,Alta,Alta,Debil,False
1,Sol,Alta,Alta,Fuerte,False
2,Nubes,Alta,Alta,Debil,True
3,Lluvia,Suave,Alta,Debil,True
4,Lluvia,Baja,Normal,Debil,True
5,Lluvia,Baja,Normal,Fuerte,False
6,Nubes,Baja,Normal,Fuerte,True
7,Sol,Suave,Alta,Debil,False
8,Sol,Baja,Normal,Debil,True
9,Lluvia,Suave,Normal,Debil,True


In [3]:
dataColores = pd.read_csv("./Colores.csv")
dataColores

,Color,Forma,Tamano,Clase
0,Rojo,Cuadrado,Grande,True
1,Azul,Cuadrado,Grande,True
2,Rojo,Redondo,Pequeno,False
3,Verde,Cuadrado,Pequeno,False
4,Rojo,Redondo,Grande,True
5,Verde,Cuadrado,Grande,False


<strong>Función total_entropy</strong> <br>
Calcula la entropia total del conjunto de datos.

![](entropy.png)

In [4]:
def total_entropy(data, label, class_list):
    total_row = data.shape[0]
    total_entr = 0
    
    for c in class_list:
        total_class_count = data[data[label] == c].shape[0]
        total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row) 
        total_entr += total_class_entr
    
    return total_entr

<strong>Función entropy</strong> <br>
Calcula la entropia de un atributo.

In [5]:
def entropy(feature_value_data, label, class_list):
    class_count = feature_value_data.shape[0]
    entropy = 0
    
    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0]
    
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count
            entropy_class = - probability_class * np.log2(probability_class) 
        
        entropy += entropy_class
        
    return entropy

<strong>Función gain</strong> <br>
Calcula la ganancia de un atributo.

![](gain.png)

In [6]:
def gain(feature_name, data, label, class_list):
    feature_list = data[feature_name].unique()
    total_row = data.shape[0]
    feature_info = 0.0
    
    for feature in feature_list:
        feature_data = data[data[feature_name] == feature]
        feature_count = feature_data.shape[0]
        feature_entropy = entropy(feature_data, label, class_list)
        feature_probability = feature_count/total_row
        feature_info += feature_probability * feature_entropy
        
    return total_entropy(data, label, class_list) - feature_info

<strong>Función make_tree</strong> <br>
Método recursivo que forma el árbol.
Encuentra el atributo con mayor ganancia, y a partir de éste, genera un sub árbol.

In [7]:
def make_tree(root, prev_feature, data, label, class_list):
    if data.shape[0] != 0:
        feature_list = data.columns.drop(label)
        max_info_gain = -1
        max_info_feature = None

        #find max info gain feature
        for feature in feature_list:
            feature_info_gain = gain(feature, data, label, class_list)
            if max_info_gain < feature_info_gain:
                max_info_gain = feature_info_gain
                max_info_feature = feature


        #generate sub tree            
        feature_count_dict = data[max_info_feature].value_counts(sort=False)
        tree = {}

        for feature, count in feature_count_dict.iteritems():
            feature_data = data[data[max_info_feature] == feature]

            assigned_to_node = False
            for c in class_list:
                class_count = feature_data[feature_data[label] == c].shape[0]

                if class_count == count:
                    tree[feature] = c
                    data = data[data[max_info_feature] != feature]
                    assigned_to_node = True
            if not assigned_to_node:
                tree[feature] = "?"

        next_root = None
        
        if prev_feature != None:
            root[prev_feature] = dict()
            root[prev_feature][max_info_feature] = tree
            next_root = root[prev_feature][max_info_feature]
        else:
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()):
            if branch == "?":
                feature_data = data[data[max_info_feature] == node]
                make_tree(next_root, node, feature_data, label, class_list)

<strong>Función id3</strong> <br>
Se indican los datos para trabajar, el atributo objetivo y encuentra los demás atributos.

In [8]:
def id3(data, label):
    tree = {}
    class_list = data[label].unique()
    make_tree(tree, None, data, label, class_list)
    
    return tree

<strong>Resultados</strong> <br>

In [9]:
tree = id3(dataTenis, 'Jugar_tenis')
tree

{'Cielo': {'Sol': {'Humedad': {'Alta': False, 'Normal': True}},
  'Nubes': True,
  'Lluvia ': {'Viento': {'Debil': True, 'Fuerte': False}}}}

![](TreeTenis.png)

In [10]:
tree = id3(dataColores, 'Clase')
tree


{'Color': {'Rojo': {'Tamano': {'Grande': True, 'Pequeno': False}},
  'Azul': True,
  'Verde': False}}

![](TreeColores.png)

<strong>Conclusión</strong> <br>
El algoritmo ID3 es sencillo de implementar que sigue un acercamiento voraz y genera un árbol de decisión que resulta útil para clasificar.